# 🤖 Complete Machine Learning Pipeline
# From Data to Distilled Models

This notebook demonstrates the complete ML pipeline:
1. **Data Preparation** - Collection, cleaning, preprocessing
2. **Tokenization** - Converting text to model-ready format
3. **Model Training** - Training base models
4. **Fine-tuning** - Advanced techniques like LoRA
5. **Knowledge Distillation** - Creating efficient models

Let's walk through each step together! 🚀

## 📦 Setup and Imports

In [ ]:
# Install required packages (run this once)
!pip install torch transformers datasets tokenizers pandas scikit-learn matplotlib seaborn tqdm peft

In [ ]:
import sys
import os

# Add src directory to path
sys.path.append('../src')

# Import our custom modules
from data_preparation import DataProcessor
from tokenization import TextTokenizer
from model_training import ModelTrainer
from fine_tuning import AdvancedFineTuner
from distillation import KnowledgeDistiller

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"🖥️ Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"✅ All imports successful!")

## 📊 Step 1: Data Preparation

First, let's prepare our data for training. This involves:
- Loading or creating a dataset
- Cleaning and preprocessing text
- Splitting into train/validation/test sets

In [ ]:
# Initialize data processor
processor = DataProcessor(data_dir="../data")

# Run the complete data preparation pipeline
dataset = processor.process_pipeline()

print("\n📋 Dataset Structure:")
print(dataset)

In [ ]:
# Let's explore our data
print("🔍 Sample Data:")
for i in range(3):
    sample = dataset['train'][i]
    print(f"\nSample {i+1}:")
    print(f"Text: {sample['text'][:100]}...")
    print(f"Label: {sample['label']} ({'Positive' if sample['label'] == 1 else 'Negative'})")

In [ ]:
# Visualize data distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Label distribution
train_labels = [sample['label'] for sample in dataset['train']]
label_counts = pd.Series(train_labels).value_counts()

ax1.pie(label_counts.values, labels=['Negative', 'Positive'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Label Distribution')

# Text length distribution
text_lengths = [len(sample['text']) for sample in dataset['train']]
ax2.hist(text_lengths, bins=30, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Text Length (characters)')
ax2.set_ylabel('Frequency')
ax2.set_title('Text Length Distribution')

plt.tight_layout()
plt.show()

print(f"📊 Average text length: {np.mean(text_lengths):.1f} characters")
print(f"📊 Dataset splits: Train={len(dataset['train'])}, Val={len(dataset['validation'])}, Test={len(dataset['test'])}")

## 🔤 Step 2: Tokenization

Now let's convert our text data into tokens that the model can understand:

In [ ]:
# Initialize tokenizer
tokenizer = TextTokenizer(
    model_name="bert-base-uncased",
    max_length=128  # Smaller for demo
)

# Show example tokenization
sample_texts = [dataset['train'][i]['text'] for i in range(3)]
tokenizer.example_tokenization(sample_texts)

In [ ]:
# Tokenize the entire dataset
tokenized_dataset = tokenizer.tokenize_dataset(dataset)

# Save tokenized dataset for later use
tokenized_dataset.save_to_disk("../data/processed/tokenized_dataset")
print("💾 Tokenized dataset saved!")

## 🤖 Step 3: Model Training

Let's train our base model:

In [ ]:
# Initialize model trainer
trainer = ModelTrainer(
    model_name="bert-base-uncased",
    num_labels=2,
    output_dir="../models/trained/bert_classifier"
)

print(f"🤖 Model initialized with {trainer.model.num_parameters():,} parameters")

In [ ]:
# Train the model (with small epochs for demo)
trained_model, results = trainer.train_model(
    tokenized_dataset,
    num_epochs=2,  # Small for demo
    batch_size=8,  # Small batch for memory
    learning_rate=2e-5
)

print("\n🎯 Training completed!")
print(f"📊 Final Results: Accuracy={results['test_results']['eval_accuracy']:.3f}, F1={results['test_results']['eval_f1']:.3f}")

In [ ]:
# Test the trained model
test_texts = [
    "This movie was absolutely amazing! I loved every minute of it.",
    "Terrible film, complete waste of time and money.",
    "It was okay, nothing special but not bad either."
]

inference_results = trainer.test_inference(test_texts)

# Visualize predictions
fig, ax = plt.subplots(figsize=(10, 6))

texts = [r['text'][:50] + '...' for r in inference_results]
confidences = [r['confidence'] for r in inference_results]
predictions = [r['predicted_class'] for r in inference_results]

colors = ['red' if p == 0 else 'green' for p in predictions]
bars = ax.barh(range(len(texts)), confidences, color=colors, alpha=0.7)

ax.set_yticks(range(len(texts)))
ax.set_yticklabels(texts)
ax.set_xlabel('Confidence')
ax.set_title('Model Predictions on Test Texts')
ax.set_xlim(0, 1)

# Add confidence values on bars
for i, (bar, conf) in enumerate(zip(bars, confidences)):
    ax.text(conf + 0.01, i, f'{conf:.3f}', va='center')

plt.tight_layout()
plt.show()

## 🔧 Step 4: Fine-tuning with LoRA

Now let's explore advanced fine-tuning techniques using LoRA (Low-Rank Adaptation):

In [ ]:
# Initialize fine-tuner
fine_tuner = AdvancedFineTuner(
    base_model_path="../models/trained/bert_classifier",
    output_dir="../models/fine_tuned"
)

print("🔧 Fine-tuner initialized!")

In [ ]:
# Compare LoRA vs Full Fine-tuning
comparison_results = fine_tuner.compare_approaches(tokenized_dataset)

# Visualize comparison
methods = list(comparison_results.keys())
accuracies = [comparison_results[m]['test_accuracy'] for m in methods]
params = [comparison_results[m]['trainable_params'] for m in methods]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy comparison
bars1 = ax1.bar(methods, accuracies, color=['skyblue', 'lightcoral'])
ax1.set_ylabel('Test Accuracy')
ax1.set_title('Accuracy Comparison')
ax1.set_ylim(0, 1)

for bar, acc in zip(bars1, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

# Parameter comparison
bars2 = ax2.bar(methods, params, color=['skyblue', 'lightcoral'])
ax2.set_ylabel('Trainable Parameters')
ax2.set_title('Trainable Parameters Comparison')
ax2.set_yscale('log')

for bar, param in zip(bars2, params):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.1, 
             f'{param:,}', ha='center', va='bottom', rotation=45)

plt.tight_layout()
plt.show()

# Calculate efficiency
param_ratio = params[0] / params[1]  # LoRA / Full
acc_ratio = accuracies[0] / accuracies[1]  # LoRA / Full

print(f"\n💡 LoRA Efficiency:")
print(f"📊 Uses {param_ratio:.1%} of full fine-tuning parameters")
print(f"📊 Achieves {acc_ratio:.1%} of full fine-tuning accuracy")
print(f"🎯 Efficiency Score: {acc_ratio/param_ratio:.1f}x more efficient!")

## 🧠 Step 5: Knowledge Distillation

Finally, let's create a smaller, faster model using knowledge distillation:

In [ ]:
# Initialize knowledge distiller
distiller = KnowledgeDistiller(
    teacher_model_path="../models/trained/bert_classifier",
    student_model_name="distilbert-base-uncased",
    output_dir="../models/distilled/distilbert_student",
    temperature=4.0,
    alpha=0.7
)

print("🧠 Knowledge distiller initialized!")
print(f"👨‍🏫 Teacher parameters: {sum(p.numel() for p in distiller.teacher_model.parameters()):,}")
print(f"👨‍🎓 Student parameters: {sum(p.numel() for p in distiller.student_model.parameters()):,}")

In [ ]:
# Perform knowledge distillation
distilled_trainer, distillation_results = distiller.distill_model(
    tokenized_dataset,
    num_epochs=3,
    batch_size=16,
    learning_rate=5e-5
)

print("\n🎯 Distillation completed!")
print(f"📊 Student Results: Accuracy={distillation_results['test_results']['eval_accuracy']:.3f}")
print(f"📦 Compression Ratio: {distillation_results['compression_ratio']:.1%}")

In [ ]:
# Compare all models
teacher_results, student_results = distiller.compare_models(tokenized_dataset)

# Test inference speed
test_texts = [
    "This movie was absolutely fantastic!",
    "I didn't like this film at all.",
    "It was an okay movie, nothing special."
]

speedup = distiller.test_inference_speed(test_texts)

print(f"\n⚡ Student model is {speedup:.2f}x faster than teacher!")

In [ ]:
# Create final comparison visualization
models = ['Teacher (BERT)', 'Student (DistilBERT)']
accuracies = [teacher_results['accuracy'], student_results['accuracy']]
parameters = [teacher_results['num_parameters'], student_results['num_parameters']]
speeds = [1.0, speedup]  # Relative speeds

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
bars1 = ax1.bar(models, accuracies, color=['navy', 'lightblue'])
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_ylim(0, 1)
for bar, acc in zip(bars1, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

# Parameters
bars2 = ax2.bar(models, [p/1e6 for p in parameters], color=['darkred', 'lightcoral'])
ax2.set_ylabel('Parameters (Millions)')
ax2.set_title('Model Size')
for bar, param in zip(bars2, parameters):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{param/1e6:.1f}M', ha='center', va='bottom')

# Speed
bars3 = ax3.bar(models, speeds, color=['purple', 'plum'])
ax3.set_ylabel('Relative Speed')
ax3.set_title('Inference Speed')
for bar, speed in zip(bars3, speeds):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             f'{speed:.1f}x', ha='center', va='bottom')

# Efficiency scatter plot
efficiency_x = [p/1e6 for p in parameters]
efficiency_y = accuracies
ax4.scatter(efficiency_x, efficiency_y, s=[100, 200], c=['red', 'green'], alpha=0.7)
ax4.set_xlabel('Parameters (Millions)')
ax4.set_ylabel('Accuracy')
ax4.set_title('Efficiency Plot')

for i, model in enumerate(models):
    ax4.annotate(model, (efficiency_x[i], efficiency_y[i]), 
                xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

# Final summary
print("\n" + "="*60)
print("🎉 COMPLETE PIPELINE SUMMARY")
print("="*60)
print(f"📊 Original Model: {parameters[0]:,} parameters, {accuracies[0]:.3f} accuracy")
print(f"⚡ Distilled Model: {parameters[1]:,} parameters, {accuracies[1]:.3f} accuracy")
print(f"📦 Size Reduction: {(1 - parameters[1]/parameters[0])*100:.1f}%")
print(f"🏃 Speed Improvement: {speedup:.1f}x faster")
print(f"🎯 Accuracy Retention: {(accuracies[1]/accuracies[0])*100:.1f}%")
print("\n✅ Pipeline completed successfully!")

## 🎯 Key Takeaways

We've successfully demonstrated the complete machine learning pipeline:

### 1. **Data Preparation** 📊
- Loaded and preprocessed text data
- Created balanced train/validation/test splits
- Analyzed data distributions

### 2. **Tokenization** 🔤
- Converted text to tokens using BERT tokenizer
- Handled padding and truncation
- Created model-ready datasets

### 3. **Model Training** 🤖
- Trained a BERT-based classifier
- Monitored training progress
- Evaluated model performance

### 4. **Fine-tuning** 🔧
- Compared LoRA vs full fine-tuning
- Demonstrated parameter efficiency
- Achieved similar performance with fewer parameters

### 5. **Knowledge Distillation** 🧠
- Created a smaller, faster student model
- Transferred knowledge from teacher to student
- Achieved significant speedup with minimal accuracy loss

### 📈 Results Summary:
- **Size Reduction**: ~66% smaller model
- **Speed Improvement**: ~2-3x faster inference
- **Accuracy Retention**: ~95% of original performance

This pipeline can be adapted for various NLP tasks and scaled for production use!